In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer
from collections import defaultdict
from typing import List, Dict, Tuple
import torch


c:\Users\brunolopez\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check if CUDA (GPU support) is available
print(torch.cuda.is_available())

# Get the name of the GPU
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3060


In [16]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

class NoteSummarizer:
    def __init__(self, notes):
        self.notes = notes
        self.tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
        self.model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

    def process(self):
        summaries = {}
        grouped_notes = self.group_by_caseID()
        for caseID, notes_text in grouped_notes.items():
            summaries[caseID] = self.summarize_notes(notes_text)
        return summaries

    def group_by_caseID(self):
        grouped_notes = {}
        for note in self.notes:
            caseID = note['caseID']
            text = note['text']
            if caseID not in grouped_notes:
                grouped_notes[caseID] = text
            else:
                grouped_notes[caseID] += ' ' + text  # Space is added to separate the text of different notes
        return grouped_notes

    def summarize_notes(self, notes_text):
        inputs = self.tokenizer([notes_text], max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = self.model.generate(inputs.input_ids, num_beams=4, min_length=30, max_length=250, length_penalty=2.0)
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary


dummy_notes = [
    {'date': '2023-10-01', 'caseID': '1', 'text': 'Initial inspection of solar PV system at Site A completed. No issues found with the PV modules or inverters.'},
    {'date': '2023-10-02', 'caseID': '1', 'text': 'Data logger setup completed at Site A. Now collecting data at 15-minute intervals. Battery storage system is operational.'},
    {'date': '2023-10-03', 'caseID': '1', 'text': 'Zero power event observed at Site A at 2:15 PM. Troubleshooting initiated to identify the cause.'},
    {'date': '2023-10-04', 'caseID': '1', 'text': 'Faulty connection found and fixed at Site A. System back online with all modules operational.'},
    
    {'date': '2023-10-01', 'caseID': '2', 'text': 'PV system installation at Site B commenced. Inverter and modules delivered. Ground mounting system installation underway.'},
    {'date': '2023-10-02', 'caseID': '2', 'text': 'Inverter setup completed at Site B. Initial testing underway. DC wiring inspection scheduled.'},
    {'date': '2023-10-03', 'caseID': '2', 'text': 'Mismatch observed in module string 3 at Site B. Reconfiguration needed to ensure optimal performance.'},
    {'date': '2023-10-04', 'caseID': '2', 'text': 'System at Site B now operational. Data collection initiated. Performance monitoring system setup completed.'},
    
    {'date': '2023-10-01', 'caseID': '3', 'text': 'Monitoring system reported underperformance at Site C. Visual inspection scheduled for tomorrow.'},
    {'date': '2023-10-02', 'caseID': '3', 'text': 'Site C visit completed. Shading issues identified due to nearby structures. Tree trimming recommended.'},
    {'date': '2023-10-03', 'caseID': '3', 'text': 'Tree trimming completed at Site C. Performance expected to improve. Data analysis ongoing.'},
    {'date': '2023-10-04', 'caseID': '3', 'text': 'Performance at Site C back to expected levels post tree trimming. Monitoring continues for any further issues.'},
    
    {'date': '2023-10-01', 'caseID': '4', 'text': 'New PV module technology discussion initiated for upcoming projects. Vendor meetings scheduled.'},
    {'date': '2023-10-02', 'caseID': '4', 'text': 'Comparative analysis of bifacial vs monofacial modules underway. Cost, performance, and longevity considered.'},
    {'date': '2023-10-03', 'caseID': '4', 'text': 'Cost-benefit analysis favors bifacial modules for next project. Procurement team informed of decision.'},
    {'date': '2023-10-04', 'caseID': '4', 'text': 'Vendor selected for supplying bifacial modules for next project. Contract negotiations initiated.'},
    
    {'date': '2023-10-01', 'caseID': '5', 'text': 'Maintenance team reported a fault in the inverter at Site D. Immediate action required.'},
    {'date': '2023-10-02', 'caseID': '5', 'text': 'Inverter replaced at Site D. System back online. Data loss during downtime is being evaluated.'},
    {'date': '2023-10-03', 'caseID': '5', 'text': 'Data recovery efforts at Site D ongoing. Performance analysis post inverter replacement scheduled.'},
    {'date': '2023-10-04', 'caseID': '5', 'text': 'Performance at Site D stable post inverter replacement. Data recovery completed. Loss minimal.'},
    
    {'date': '2023-10-01', 'caseID': 'PM1', 'text': 'Preventative Maintenance scheduled at Site A. Checklist prepared for routine inspection.'},
    {'date': '2023-10-02', 'caseID': 'PM1', 'text': 'PM completed at Site A. All systems operational. No issues found.'},
    {'date': '2023-10-03', 'caseID': 'PM1', 'text': 'PM report filed for Site A. Next PM scheduled in 6 months.'},
    
    {'date': '2023-10-01', 'caseID': 'PM2', 'text': 'Preventative Maintenance scheduled at Site B. Equipment list reviewed.'},
    {'date': '2023-10-02', 'caseID': 'PM2', 'text': 'PM completed at Site B. Minor adjustments made to the tracking system.'},
    {'date': '2023-10-03', 'caseID': 'PM2', 'text': 'PM report filed for Site B. Tracking system performance monitored.'},
    
    {'date': '2023-10-01', 'caseID': 'random', 'text': 'The quick brown fox jumps over the lazy dog.'},
    {'date': '2023-10-02', 'caseID': 'random', 'text': 'Lorem ipsum dolor sit amet, consectetur adipiscing elit.'},
    {'date': '2023-10-03', 'caseID': 'random', 'text': 'A journey '}
]

In [17]:
# Usage:
summarizer = NoteSummarizer(dummy_notes)
summaries = summarizer.process()
print(summaries)

{'1': 'Initial inspection of solar PV system at Site A completed. No issues found with the PV modules or inverters. Faulty connection found and fixed. System back online with all modules operational.', '2': 'PV system installation at Site B commenced. Inverter and modules delivered. Ground mounting system installation underway. Initial testing underway. DC wiring inspection scheduled. Mismatch observed in module string 3.', '3': 'Monitoring system reported underperformance at Site C. Site C visit completed. Shading issues identified due to nearby structures. Tree trimming recommended.', '4': 'New PV module technology discussion initiated for upcoming projects. Comparative analysis of bifacial vs monofacial modules underway. Cost, performance, and longevity considered. Procurement team informed of decision.', '5': 'Maintenance team reported a fault in the inverter at Site D. Immediate action required. Inverter replaced atSite D. System back online. Data loss during downtime being evalua